In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

data_de_hoje = datetime.date.today()

# Coleta os produtos da loja
produtos = []

In [8]:
# Variáveis
service = ChromeService(
    ChromeDriverManager(driver_version="137.0.7151.104").install()
)
driver = webdriver.Chrome(service=service)

driver.get("https://mercado.carrefour.com.br/")

time.sleep(1)

precos_carrefour = []


# Busca pelo nome do producto
search = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Pesquise por produtos ou marcas']"))
)
search.click()
search.send_keys("salmao")
search.send_keys(Keys.RETURN)

In [9]:
search = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
)
search.click()

# Reire na loja
retire = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
)
retire.click()

# Seleciona a cidade (exemplo: primeira cidade válida)
select = Select(WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
))

total_cidades = len(select.options) - 1  # Subtrai 1 se quiser ignorar a primeira opção ("Selecionar cidade")
print(f"[INFO] Total de cidades disponíveis: {total_cidades}")



[INFO] Total de cidades disponíveis: 56


In [15]:
for i in range(41, 55):
    #### 2 - MANAUS ####
    select.select_by_index(i)  # Exemplo: primeira cidade após o "Selecionar cidade"

    # Aguarda até que as lojas da cidade carreguem no DOM
    loja_elements = WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4"))
    )
    # Conta e informa o total de lojas
    total_lojas = len(loja_elements)
    print(f"[INFO] Total de lojas na cidade {i}: {total_lojas}")



    for loja_index in range(len(loja_elements)):
        # Recarrega a lista de lojas antes de cada clique
        lojas_atualizadas = driver.find_elements(By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4")
        time.sleep(1)
        loja = lojas_atualizadas[loja_index]
        nome_loja = loja.text.strip()
        print(f"[Cidade {i}] Loja selecionada: {nome_loja}")
        loja.click()
        time.sleep(3)


        # Scroll para carregar todos os produtos
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        # Captura os blocos de produto
        divs_produtos = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")

        print(f"[INFO] Total de blocos de produto encontrados: {len(divs_produtos)}")

        for idx in range(len(divs_produtos)):
            try:
                # Recarrega a lista para evitar StaleElementReferenceException
                divs_produtos_atualizados = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")
                div = divs_produtos_atualizados[idx]

                texto_produto = div.text.strip()

                if texto_produto:
                    # Formata igual ao outro código: produto + loja + cidade
                    registro = f"{texto_produto}\n{nome_loja}"
                    produtos.append(registro)

            except Exception as e:
                print(f"[WARNING] Erro ao coletar produto {idx+1}: {e}")
                continue

        print(f"[INFO] Total de produtos coletados até agora: {len(produtos)}")



        # Antes de voltar para a próxima loja:
        driver.back()
        time.sleep(3)

        # Agora, reabrir o picker de cidade
        # Faz um clique genérico no body para destravar o picker
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class*='container-default']"))
        ).click()
        time.sleep(1)
        
        # Busca por cidade
        search = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
        )
        search.click()
        
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
        ).click()

        # E aqui novamente... recaptura o Select ANTES de re-selecionar a cidade!
        select = Select(WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
        ))
        #select.select_by_index(cidade_index)

        
        select.select_by_index(i)

        time.sleep(2)


driver.quit()

    

[INFO] Total de lojas na cidade 41: 1
[Cidade 41] Loja selecionada: Hiper Nova Piracicaba
Avenida Ruy Teixeira Mendes
Glebas Califórnia, Piracicaba - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1135
[INFO] Total de lojas na cidade 42: 1
[Cidade 42] Loja selecionada: Bairro Praia Grande
Avenida Castelo Branco
Vila Guilhermina, Praia Grande - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1150
[INFO] Total de lojas na cidade 43: 1
[Cidade 43] Loja selecionada: Hiper Prudente
Avenida Manoel Goulart
Jardim das Rosas, Presidente Prudente - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos coletados até agora: 1165
[INFO] Total de lojas na cidade 44: 3
[Cidade 44] Loja selecionada: Hiper Ribeirão Shopping
Avenida Coronel Fernando Ferreira Leite
Jardim California, Ribeirão Preto - SP
[INFO] Total de blocos de produto encontrados: 15
[INFO] Total de produtos co

In [16]:
i

54

In [17]:
produtos

['Salmão Inteiro Fresco Aprox. 4Kg\nR$ 359,96\nR$ 89,99/kg\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Salmão Inteiro Congelado Aprox. 2 kg\nR$ 144,18\nR$ 72,09/kg\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Salmão em Pedaços Carrefour Classic 500g\nR$ 89,99\n-16%\nR$ 75,98\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Filé de Salmão Carrefour Classic 500g\nR$ 93,59\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Ração Úmida para Gato Purina Friskies Salmão com Molho Sachê 85g\nR$ 3,45\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeitona Verde Recheada Especial com Salmão Vale Fértil 130g\nR$ 19,98\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Petisco para Gatos Adultos Dreamies Ir

In [18]:
len(produtos)

1734

In [19]:
produtos_limpos = []

for p in produtos:
    # Remover linha se for "Produto indisponível" ou "lencol"
    if 'Produto indisponível' in p or 'lencol' in p.lower():
        continue

    # Remover a palavra "ADICIONAR"
    p = p.replace('ADICIONAR', '').strip()

    # Remover qualquer preço no formato por kg, litro, etc
    p = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", p, flags=re.IGNORECASE).strip()

    # Se sobrar algo válido, incluir na lista final
    if p:
        produtos_limpos.append(p)

produtos_limpos

['Salmão Inteiro Fresco Aprox. 4Kg\nR$ 359,96\n\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Salmão Inteiro Congelado Aprox. 2 kg\nR$ 144,18\n\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Salmão em Pedaços Carrefour Classic 500g\nR$ 89,99\n-16%\nR$ 75,98\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Filé de Salmão Carrefour Classic 500g\nR$ 93,59\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Ração Úmida para Gato Purina Friskies Salmão com Molho Sachê 85g\nR$ 3,45\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeitona Verde Recheada Especial com Salmão Vale Fértil 130g\nR$ 19,98\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Petisco para Gatos Adultos Dreamies Irresistível Sabor Salmão 40g\nR$ 7,85\n\nHiper Maceió Jatiúca\nAvenida Comend

In [20]:
len(produtos_limpos)

1623

In [21]:
# Lista original
dados_brutos = produtos_limpos

data_de_hoje = data_de_hoje
dados_estruturados = []

for linha in dados_brutos:
    try:
        partes = linha.split("\n\n")
        produto_bloco = partes[0]
        endereco_bloco = partes[1] if len(partes) > 1 else ""

        # Quebra o bloco do produto em linhas individuais
        linhas = produto_bloco.strip().split("\n")

        # REMOVE TRECHOS tipo "/kg", "/g", "/L" apenas de dentro das linhas
        linhas_limpa = []
        for l in linhas:
            # Remove apenas os fragmentos como "R$ XX,XX/kg", "R$ XX,XX/L", etc
            l_limpo = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", l, flags=re.IGNORECASE)
            linhas_limpa.append(l_limpo.strip())

        # Nome do produto
        produto_nome = next((x for x in linhas_limpa if "R$" not in x and "%" not in x and "promoção" not in x.lower()), "").strip()

        # Preço final (último R$ válido depois da limpeza)
        precos_encontrados = [x for x in linhas_limpa if re.search(r"R\$ ?\d+,\d{2}", x)]
        preco_final = precos_encontrados[-1] if precos_encontrados else None
        if preco_final:
            preco_final = re.search(r"R\$ ?(\d+,\d{2})", preco_final).group(1)
            preco_final = float(preco_final.replace(",", "."))

        # Desconto
        desconto_linha = next((x for x in linhas_limpa if "%" in x and "-" in x), None)
        desconto = float(desconto_linha.replace("%", "").replace(",", ".").strip()) if desconto_linha else None

        # Loja e endereço
        endereco_linhas = endereco_bloco.strip().split("\n")
        loja = endereco_linhas[0] if len(endereco_linhas) > 0 else ""
        endereco = " - ".join(endereco_linhas[1:]) if len(endereco_linhas) > 1 else ""

        # Cidade e estado
        cidade_estado_texto = endereco_linhas[-1] if len(endereco_linhas) >= 1 else ""
        if "," in cidade_estado_texto and " - " in cidade_estado_texto:
            cidade_texto = cidade_estado_texto.split(",")[-1].strip()
            cidade, estado = cidade_texto.split(" - ")
        else:
            cidade, estado = "", ""

        # Monta o dicionário final
        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco_bloco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip(),
            "data": data_de_hoje
        })

    except Exception as e:
        print(f"[ERRO] Linha com problema:\n{linha}\n-> {e}")

# DataFrame final
df_produtos = pd.DataFrame(dados_estruturados)

# Garante que a coluna 'data' exista antes de reorganizar
if 'data' not in df_produtos.columns:
    df_produtos['data'] = data_de_hoje

# Reorganizar com 'data' na frente
colunas = ['data'] + [col for col in df_produtos.columns if col != 'data']
df_produtos = df_produtos[colunas]

#Incluir coluna de regioes
regioes_brasil = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste',
    'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}
df_produtos['regiao'] = df_produtos['estado'].map(regioes_brasil).fillna('Desconhecida')

# Alterar os dados da coluna cidade
df_produtos['cidade'] = df_produtos['cidade']+'/'+df_produtos['estado']

df_produtos.rename(columns={'estado': 'uf'}, inplace=True)

df_produtos

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-06-24,Salmão Inteiro Fresco Aprox. 4Kg,359.96,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-06-24,Salmão Inteiro Congelado Aprox. 2 kg,144.18,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-06-24,Salmão em Pedaços Carrefour Classic 500g,75.98,-16.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-06-24,Filé de Salmão Carrefour Classic 500g,93.59,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-06-24,Ração Úmida para Gato Purina Friskies Salmão c...,3.45,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
...,...,...,...,...,...,...,...,...,...
1618,2025-06-24,Ração Úmida para Cachorro Adulto Pequeno Purin...,4.38,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
1619,2025-06-24,Petisco para Gatos Adultos Dreamies Irresistív...,5.99,-20.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
1620,2025-06-24,Ração Seca para Gatos Castrados Purina One Fra...,34.99,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
1621,2025-06-24,Ração Úmida Sheba Sachê Cortes Selecionados Sa...,4.09,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste


In [22]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1623 non-null   object 
 1   produto   1623 non-null   object 
 2   preco     1623 non-null   float64
 3   desconto  202 non-null    float64
 4   loja      1623 non-null   object 
 5   endereco  1623 non-null   object 
 6   cidade    1623 non-null   object 
 7   uf        1623 non-null   object 
 8   regiao    1623 non-null   object 
dtypes: float64(2), object(7)
memory usage: 114.2+ KB


In [23]:
df_produtos.to_csv('precos_carrefour_salmao_20250624.csv', index=False)

In [1]:
import pandas as pd

In [2]:
df_produtos = pd.read_csv('precos_carrefour_salmao_20250624.csv')

In [3]:
df_produtos.head(5)

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-06-24,Salmão Inteiro Fresco Aprox. 4Kg,359.96,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-06-24,Salmão Inteiro Congelado Aprox. 2 kg,144.18,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-06-24,Salmão em Pedaços Carrefour Classic 500g,75.98,-16.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-06-24,Filé de Salmão Carrefour Classic 500g,93.59,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-06-24,Ração Úmida para Gato Purina Friskies Salmão c...,3.45,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste


In [4]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1623 non-null   object 
 1   produto   1604 non-null   object 
 2   preco     1623 non-null   float64
 3   desconto  202 non-null    float64
 4   loja      1623 non-null   object 
 5   endereco  1623 non-null   object 
 6   cidade    1623 non-null   object 
 7   uf        1623 non-null   object 
 8   regiao    1623 non-null   object 
dtypes: float64(2), object(7)
memory usage: 114.2+ KB


In [5]:
df_produtos.loja.unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [6]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')
df_lojas_geo['loja'].unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [7]:
# Comparar lojas presentes em df_produtos mas não em df_lojas_geo
lojas_produtos = set(df_produtos.loja.unique())
lojas_geo = set(df_lojas_geo['loja'].unique())

lojas_somente_produtos = lojas_produtos - lojas_geo
lojas_somente_geo = lojas_geo - lojas_produtos

print("Lojas em df_produtos e não em df_lojas_geo:", lojas_somente_produtos)
print("Lojas em df_lojas_geo e não em df_produtos:", lojas_somente_geo)

Lojas em df_produtos e não em df_lojas_geo: {'Hiper Tietê', 'Hiper Novo Hamburgo', 'Bairro Caçapava'}
Lojas em df_lojas_geo e não em df_produtos: {'Hiper Vila Maria', 'Bairro Vinhedo', 'Hiper Taubaté Charles', 'Bairro Taubaté'}


In [8]:
# Nomes de lojas duplicadas
df_lojas_geo['loja'].value_counts().loc[lambda x: x > 1]


loja
Hiper Guarulhos       3
Hiper Osasco          3
Hiper Santa Maria     2
Hiper Campo Grande    2
Name: count, dtype: int64

In [9]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')

# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

df_geo_merge['lat'] = df_geo_merge['lat'].astype(str).str.replace(',', '.').astype(float)
df_geo_merge['long'] = df_geo_merge['long'].astype(str).str.replace(',', '.').astype(float)

# Identificar lojas sem geolocalização
lojas_sem_geo = df_geo_merge[df_geo_merge['lat'].isna()]['loja'].unique()

# Remover linhas com qualquer NaN
df_geo_merge = df_geo_merge[df_geo_merge['lat'].notna()]
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

lojas_sem_geo.tolist()

['Hiper Novo Hamburgo', 'Bairro Caçapava', 'Hiper Tietê']

In [71]:
# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

In [10]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1623 non-null   object 
 1   produto   1604 non-null   object 
 2   preco     1623 non-null   float64
 3   desconto  202 non-null    float64
 4   loja      1623 non-null   object 
 5   endereco  1623 non-null   object 
 6   cidade    1623 non-null   object 
 7   uf        1623 non-null   object 
 8   regiao    1623 non-null   object 
dtypes: float64(2), object(7)
memory usage: 114.2+ KB


In [ ]:
df_produtos.loc[df_produtos['loja']== 'Hiper Águas Claras']

In [11]:
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

In [12]:
df_geo_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1739 entries, 0 to 1855
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      1739 non-null   object 
 1   produto   1720 non-null   object 
 2   preco     1739 non-null   float64
 3   desconto  219 non-null    float64
 4   loja      1739 non-null   object 
 5   endereco  1739 non-null   object 
 6   cidade    1739 non-null   object 
 7   uf        1739 non-null   object 
 8   regiao    1739 non-null   object 
 9   lat       1739 non-null   float64
 10  long      1739 non-null   float64
dtypes: float64(4), object(7)
memory usage: 163.0+ KB


In [13]:
df_geo_merge

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao,lat,long
0,2025-06-24,Salmão Inteiro Fresco Aprox. 4Kg,359.96,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
1,2025-06-24,Salmão Inteiro Congelado Aprox. 2 kg,144.18,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
2,2025-06-24,Salmão em Pedaços Carrefour Classic 500g,75.98,-16.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
3,2025-06-24,Filé de Salmão Carrefour Classic 500g,93.59,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
4,2025-06-24,Ração Úmida para Gato Purina Friskies Salmão c...,3.45,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,-9.64959,-35.71739
...,...,...,...,...,...,...,...,...,...,...,...
1851,2025-06-24,Ração Úmida para Cachorro Adulto Pequeno Purin...,4.38,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,-23.48623,-47.47117
1852,2025-06-24,Petisco para Gatos Adultos Dreamies Irresistív...,5.99,-20.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,-23.48623,-47.47117
1853,2025-06-24,Ração Seca para Gatos Castrados Purina One Fra...,34.99,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,-23.48623,-47.47117
1854,2025-06-24,Ração Úmida Sheba Sachê Cortes Selecionados Sa...,4.09,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,-23.48623,-47.47117


In [ ]:
df_geo_merge.loc[df_geo_merge['cidade']== 'Brasília/DF']

In [14]:
df_geo_merge.to_csv('precos_carrefour_salmao_20250624_loc.csv', index=False)